# Model Total Ionization Dose (TID) example notebook

This notebook showcases how to model **Total Ionization Dose (TID)** inside `PASEOS`. To this aim, we will use a **custom property** to model the accumulated dose and the consequent risk of failure. <br>
We used [SPENVIS](https://www.spenvis.oma.be/) to estimate the total dose and the electron and protons fluxes over 30 mission days on the [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) sun-synchronous orbit. 

In [ ]:
%load_ext autoreload
%autoreload 2
import sys 
import os
import asyncio
sys.path.insert(1, os.path.join("..",".."))
import pykep as pk
import numpy as np
import paseos
from paseos import ActorBuilder, SpacecraftActor
from paseos.utils.load_default_cfg import load_default_cfg
from numpy.random import lognormal
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches

# 1) - Instantiate a space actor

First of all, let's create the scaffolds for  our satellite **sat** over Sentinel-2B orbit. To know more about scaffolds and actors, please check our [Sentinel-2B example notebook](https://github.com/aidotse/PASEOS/blob/main/examples/Sentinel_2_example_notebook/Sentinel2_example_notebook.ipynb).

In [ ]:
# Define local actor
sat = ActorBuilder.get_actor_scaffold(name="sat", actor_type=SpacecraftActor, epoch=pk.epoch(0))

#### 1.a) - Add an orbit for S2B

Since **sat** is orbiting around Earth, let's define `earth` as `pykep.planet` object. 

Internally, paseos uses pykep.planet objects to model gravitational acceleration and Keplerian orbits.

In [ ]:
# Define central body
earth = pk.planet.jpl_lp("earth")

To find realistic orbits for **sat**, we can exploit `Two Line Elements (TLEs)` (Downloaded on 27-10-2022). This would allow finding their ephemerides at time = 27-10-2022 12:00:00.
Please, refer to [Two-line_element_set](https://en.wikipedia.org/wiki/Two-line_element_set) to know more about TLEs.

In [ ]:
#Define today as pykep epoch (27-10-22)
#please, refer to https://esa.github.io/pykep/documentation/core.html#pykep.epoch
today = pk.epoch_from_string('2022-10-27 12:00:00.000')

sentinel2B_line1 = "1 42063U 17013A   22300.18652110  .00000099  00000+0  54271-4 0  9998"
sentinel2B_line2 = "2 42063  98.5693  13.0364 0001083 104.3232 255.8080 14.30819357294601"
sentinel2B = pk.planet.tle(sentinel2B_line1, sentinel2B_line2)

#Calculating S2B ephemerides.
sentinel2B_eph=sentinel2B.eph(today)


Now we define the actor's orbit for **S2B**.

In [ ]:
#Adding orbit around Earth based on previously calculated ephemerides
ActorBuilder.set_orbit(actor=sat, 
                       position=sentinel2B_eph[0], 
                       velocity=sentinel2B_eph[1], 
                       epoch=today, central_body=earth)

# 2) - Instantiate PASEOS simulation

To instantiate `PASEOS`, we consider **S2B** as `local_actor`. The initial time is set to `today`.

In [ ]:
cfg=load_default_cfg() # loading cfg to modify defaults
cfg.sim.start_time=today.mjd2000 * pk.DAY2SEC # convert epoch to seconds
cfg.sim.activity_timestep = 10.0 # update frequency of the constraint function. 
sim = paseos.init_sim(sat, cfg)

# 3) - Modelling TID effects in PASEOS

According to SPENVIS the total dose over 30 mission days is of 3.165E+03 rad (electrons and protons). This is calculated by using 2.5mm spherical shielding wrapping the electronic device.

In [ ]:
dose_krad=3.165 

We assume now that the dose increases linearly between t=0 to t=30 days. 

In [ ]:
d_dose_dt=dose_krad / (30 * 24 * 3600)

To model TID in PASEOS, we can use custom properties. First of all, we define the a property called `total_ionizing_dose`. We then create a dedicated an update function `prop_update_fn` to compute the dose over time.

In [ ]:
# Add a custom property which tracks the TID
prop_name = "total_ionizing_dose"
# Define the update function
def prop_update_fn(actor, dt, power_consumption):
    return actor.get_custom_property(prop_name) + d_dose_dt * dt

We can now add a custom property that models the `total_ionizing_dose`, which is updated as described by `prop_update_fn`.

In [ ]:
ActorBuilder.add_custom_property(actor=sat, property_name=prop_name, update_function=prop_update_fn, initial_value=0.0)

# 4) - Calculate the probability of a failure due to the TID

According to the work [Wang, Jian-zhao et al.](https://www.sciencedirect.com/science/article/pii/S0026271422002712?casa_token=PbtCHwrwtKwAAAAA:hddUtlxhJM_CC-nUZBuxG2qZ7793olmDjTO7ZyztDxjjsteCl1fxvlh88SjEiRIeGVR430Po4EJW), the failure probability due to TID ($P_{failure}$) can be calculates as follows: 

<br> $P_{failure}=\int_{0}^{dose(t)} (1-H(x)) \cdot g(x) \cdot dx $

where:

- $g(x)$ is the Probability Distribution Function of the devices' failure doses.
- $H(x)$ is the cumulative distribution function of radiation dose from space during the mission period. 

To estimate $H(x)$, we centered a lognormal distribution in the value of the dose. Then, we calculated its cumulative distributed function. <br> To estimate the variability of the curve as defined in [Wang, Jian-zhao et al.](https://www.sciencedirect.com/science/article/pii/S0026271422002712?casa_token=PbtCHwrwtKwAAAAA:hddUtlxhJM_CC-nUZBuxG2qZ7793olmDjTO7ZyztDxjjsteCl1fxvlh88SjEiRIeGVR430Po4EJW), we used as sample the values reported in the sun-synchronous orbit for 100 mils reported in [Xapsos, M. A., et al.](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7563310).

In [ ]:
def generate_lognormal_pdf_and_cdf(x_mean, x_sigma, n_samples):
    x=lognormal(x_mean,  x_sigma , size=n_samples)
    count, bins_count = np.histogram(x, bins=n_samples)
    count_max=np.max(count)
    bins_count=bins_count/bins_count[np.where(count==count_max)][0] * x_mean
    bins_count=bins_count[1:]
        
    # finding the PDF of the histogram using count values

    # using numpy np.cumsum to calculate the CDF
    # We can also find using the PDF values by looping and adding

    pdf = count/(sum(count))
    cdf_area = np.trapz(pdf, bins_count)
    pdf=pdf/cdf_area
    cdf=np.zeros_like(pdf)
    for n in range(len(cdf)):
        cdf[n]=np.trapz(pdf[:n], bins_count[:n])
    
    return bins_count, cdf, pdf

In [ ]:
plt.figure(0)
bins_count_H, cdf, pdf = generate_lognormal_pdf_and_cdf(dose_krad, 0.15, 10000)
H= dict(zip(bins_count_H, cdf))
#plt.semilogx(bins_count, pdf, color="red", label="PDF")
plt.semilogx(bins_count_H, cdf, label="H(dose)")
plt.legend()
plt.show()

$g(x)$ shall be specified by the user depending on the target component.  We used a lognormal CDF centered on the average TID value. <br> For this example, we use an unreasonably low TID value (i.e., 0.001 krad) to possibly experiment a failure during the mission in a short time.

In [ ]:
TID_krad = 0.001
bins_count_g, cdf, pdf = generate_lognormal_pdf_and_cdf(TID, TID/2, 10000)
g= dict(zip(bins_count_g, pdf))

plt.close()
plt.figure(1)
plt.semilogx(bins_count_g, pdf, color="red", label="g(dose)")
plt.legend()
plt.show()

Extending $g(dose)$ and $H(dose)$ on a common unified domain.

In [ ]:
unified_domain=np.union1d(bins_count_H, bins_count_g)
H_extended=dict(zip(unified_domain, np.zeros_like(unified_domain)))
g_extended=dict(zip(unified_domain, np.zeros_like(unified_domain)))

for x in bins_count_g:
    g_extended[x]=g[x]
else:
    g_extended[x]=0

        
for x in bins_count_H:
    H_extended[x]=H[x]
else:
    H_extended[x]=0


We can now generate a constraint function that checks if a failure event happens.

In [ ]:
# Plot current status of PASEOS and get a plotter
plotter = paseos.plot(sim, paseos.PlotType.SpacePlot)
    
def constraint_func_TID_failure():
    dose=sat.get_custom_property("total_ionizing_dose")
    
    #Finding the closest domain value to the dose
    dose_nearest_idx = (np.abs(unified_domain - dose)).argmin()
    
    #Calculating the probability 
    y=np.zeros_like(unified_domain)
    
    for n in range(dose_nearest_idx):
        y[n]=(1 - H_extended[unified_domain[n]]) * g_extended[unified_domain[n]]
    p=np.trapz(y, unified_domain)
    
    
    #Sampling uniform distribution probability 
    p_s = np.random.uniform(0, 1.0)
    
    
    if (p_s <= p):
        print("Detected fault due to TID!")
        print("Damage probability: ", p)
        print("Dose [krad]: ", dose)
        print("Time:", sat.local_time)
        return False
    else:
        return True


# 5) - Let's test it out

Let's advance the time of 30 days and check if an interrupt happens.

In [ ]:
sim.advance_time(3600 * 24 * 30, 10, constraint_function=constraint_func_TID_failure)